In [1]:
from tms_thresholds.threshold_sim import estimate_cell_threshold, cell_type_threshold_map

In [2]:
# Monophasic
tms_pulse_shape='Monophasic'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_TTPC2_cADpyr'
morphIDs = [1]#, 2, 3, 4, 5]
angular_resolution = 5  # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 200 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 1666/1666 [26:26<00:00,  1.05it/s]

{'L5_TTPC2_cADpyr_1': {'Polar': {np.float64(0.0): {'Azimuthal': {0.0: {'threshold': 296.875, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}}}, np.float64(5.0): {'Azimuthal': {0.0: {'threshold': 298.4375, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 51.42857142857143: {'threshold': 293.75, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 102.85714285714286: {'threshold': 290.625, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 154.28571428571428: {'threshold': 292.1875, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 205.71428571428572: {'threshold': 298.4375, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 257.14285714285717: {'threshold': 303.125, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 308.57142857142856: {'threshold': 303.125, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}}}, np.float64(10.0): {'Azimuthal': {0.0: {'threshol

In [ ]:
# Half-Sine
tms_pulse_shape='Half-Sine'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 2 # Actual pulse width required for input of half-sine pulse (due to half-sine pulse width definition not including second negative phase)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_TTPC2_cADpyr'
morphIDs = [1, 2, 3, 4, 5]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 2110/2110 [29:57<00:00,  1.17it/s] 

{'L5_TTPC2_cADpyr_1': {'Polar': {0.0: {'Azimuthal': {0.0: {'threshold': 217.1875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 10.0: {'Azimuthal': {0.0: {'threshold': 223.4375, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 51.42857142857143: {'threshold': 214.0625, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 102.85714285714286: {'threshold': 210.9375, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 154.28571428571428: {'threshold': 212.5, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 205.71428571428572: {'threshold': 221.875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 257.14285714285717: {'threshold': 229.6875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 308.57142857142856: {'threshold': 229.6875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 20.0: {'Azimuthal': {0.0: {'threshold': 226.5625, 'num_sim_bo

In [6]:
# Biphasic
tms_pulse_shape='Biphasic'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 4 # Actual pulse width required for input of biphasic pulse (due to biphasic pulse width definition not including subsequent phases)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_TTPC2_cADpyr'
morphIDs = [1, 2, 3, 4, 5]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 2110/2110 [33:38<00:00,  1.05it/s] 

{'L5_TTPC2_cADpyr_1': {'Polar': {0.0: {'Azimuthal': {0.0: {'threshold': 155.46875, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}}}, 10.0: {'Azimuthal': {0.0: {'threshold': 164.0625, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}, 51.42857142857143: {'threshold': 165.625, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}, 102.85714285714286: {'threshold': 162.5, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}, 154.28571428571428: {'threshold': 155.46875, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 205.71428571428572: {'threshold': 150.78125, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 257.14285714285717: {'threshold': 151.5625, 'num_sim_bounding': 2, 'num_sim_refining': 7, 'num_sim_total': 9}, 308.57142857142856: {'threshold': 157.8125, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}}}, 20.0: {'Azimuthal': {0.0: {'threshold': 178.125, 'num_sim_bounding

In [ ]:
# Monophasic
tms_pulse_shape='Monophasic'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 4 # Actual pulse width required for input of monophasic pulse (due to monophasic pulse width definition not including trailing negative phase)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_LBC_cNAC'
morphIDs = [1, 2, 3, 4, 5]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 2110/2110 [32:48<00:00,  1.07it/s] 

{'L5_LBC_cNAC_1': {'Polar': {0.0: {'Azimuthal': {0.0: {'threshold': 287.5, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 10.0: {'Azimuthal': {0.0: {'threshold': 265.625, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 51.42857142857143: {'threshold': 271.875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 102.85714285714286: {'threshold': 293.75, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 154.28571428571428: {'threshold': 298.4375, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 205.71428571428572: {'threshold': 281.25, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 257.14285714285717: {'threshold': 281.25, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 308.57142857142856: {'threshold': 278.125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 20.0: {'Azimuthal': {0.0: {'threshold': 253.125, 'num_sim_bounding': 3, '

In [ ]:
# Half-Sine
tms_pulse_shape='Half-Sine'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 2 # Actual pulse width required for input of half-sine pulse (due to half-sine pulse width definition not including second negative phase)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_LBC_cNAC'
morphIDs = [1, 2, 3, 4, 5]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 2110/2110 [33:50<00:00,  1.04it/s] 


{'L5_LBC_cNAC_1': {'Polar': {0.0: {'Azimuthal': {0.0: {'threshold': 285.9375, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 10.0: {'Azimuthal': {0.0: {'threshold': 262.5, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 51.42857142857143: {'threshold': 268.75, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 102.85714285714286: {'threshold': 290.625, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 154.28571428571428: {'threshold': 293.75, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 205.71428571428572: {'threshold': 278.125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 257.14285714285717: {'threshold': 276.5625, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 308.57142857142856: {'threshold': 276.5625, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 20.0: {'Azimuthal': {0.0: {'threshold': 251.5625, 'num_sim_bounding': 

In [ ]:
# Biphasic
tms_pulse_shape='Biphasic'
tms_pulse_width_ms=0.075 # Duration of the positive phase of the pulse
tms_pulse_width_ms *= 4 # Actual pulse width required for input of biphasic pulse (due to biphasic pulse width definition not including subsequent phases)

pre_tms_period = 0 # For cell to reach baseline/get data on baseline activity
spike_detection_time = 1
tms_spiking_period = tms_pulse_width_ms+spike_detection_time # For spike detection

# Cell type descriptions from the Blue Brain Project glossary: https://bbp.epfl.ch/nmc-portal/glossary.html
# Layer 5 thick-tufted pyramidal cell with an early bifurcating apical tuft - continuous accommodating (adapting) pyramidal cell
cell_name = 'L5_LBC_cNAC'
morphIDs = [1, 2, 3, 4, 5]
angular_resolution = 10 # Angular degrees of separation to use when when determining which field directions to simulate
                        # Applies to both the polar and azimuthal angles

duration = pre_tms_period + tms_spiking_period # ms

starting_E = 100 # Starting point of E-field amplitude in V/m to perform binary search from to estimate the cell's firing threshold at each parameter set
search_factor = 2   # Factor to scale by (up or down) when searching for the bounds to perform the binary search within
                    # Factor should be lower if guess is good to reflect confidence and minimize num of simulations
# How to determine the optimal starting_E and search_factor? Do they relate to the mean/median & stdev/sterr of the distribution of thresholds across the population of cells?

search_precision = 0.01 # Threshold estimation will be at most this amount of relative error above the true threshold (e.g. 0.01 = 1% error)

num_cores = 8 # Number of cores to allocate to the batch of parallel simulations

tms_params = dict(
    # Coupling params
    decay_rate_percent_per_mm=0,            # Rate of decay that the field diminishes by over space (uniform field at 0)
    E_field_dir={},                         # Empty because it will be populated and iterated over in cell_type_threshold_map
    decay_dir={'Coord_type': 'Spherical',   # Direction over which the field decays (meaningless when decay is 0)
                'Polar': 180,
                'Azimuthal': 0,},
    somatodendritic_axis=[0, 1, 0],         # Defines the direction that Polar=0 points in
    # Waveform params
    stim_type='sTMS',                       # Single-pulse TMS
    stim_start_ms=pre_tms_period,           # When to apply the pulse
    num_pulses_per_burst=1,                 # Number of pulses in a burst (useful for theta burst stimulation)
    tms_pulse_width_ms=tms_pulse_width_ms,
    tms_pulse_shape=tms_pulse_shape,
    # Simulation params
    tms_spiking_period=tms_spiking_period,  # Only pass this in for saving at the end
    simulation_duration_ms=duration,
    default_dt = 0.025,                     # Time step when outside of any pulses
    num_time_steps_in_pulse = 50,           # Number of time steps to spend on each pulse (such that pulse_dt = tms_pulse_width / num_time_steps_in_pulse)
    # Plotting quasipotentials
    plot=False,
)

# Synaptic parameters (not fully implemented)
syn_params = None

# Runs the simulations, saves the results in the data folder, and returns the threshold_map data structure
# Takes ~30 minutes to run on 8 cores on my desktop
threshold_map = cell_type_threshold_map(cell_name, morphIDs, starting_E, search_factor, search_precision, angular_resolution, 
                                            num_cores, tms_params, syn_params, ecs_spike_recording=True, save_results=f"{cell_name}_{tms_pulse_shape}")
print(threshold_map)

100%|██████████| 2110/2110 [37:02<00:00,  1.05s/it] 

{'L5_LBC_cNAC_1': {'Polar': {0.0: {'Azimuthal': {0.0: {'threshold': 210.9375, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}}}, 10.0: {'Azimuthal': {0.0: {'threshold': 200, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}, 51.42857142857143: {'threshold': 217.1875, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 102.85714285714286: {'threshold': 228.125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 154.28571428571428: {'threshold': 220.3125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 205.71428571428572: {'threshold': 207.8125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 257.14285714285717: {'threshold': 203.125, 'num_sim_bounding': 3, 'num_sim_refining': 7, 'num_sim_total': 10}, 308.57142857142856: {'threshold': 195.3125, 'num_sim_bounding': 2, 'num_sim_refining': 6, 'num_sim_total': 8}}}, 20.0: {'Azimuthal': {0.0: {'threshold': 195.3125, 'num_sim_bounding': 

: 